In [1]:
import numpy as np
from PIL import Image       
import math

y_train = np.load("orientations_train.npy")

train_size = 10000
image_length = 4096

def vectorize_images(filename, data_size):
    # vectorized 10000 images
    X = np.empty(shape=(data_size, image_length))

    for i in range(data_size):
        img_rgb = Image.open("{filename}/{i}.jpg".format(filename = filename, i = i))
        img_gray = img_rgb.convert('L')

        np_img = np.array(img_gray)
        X[i] = np_img.flatten()
        
    return X

X_train = vectorize_images("3dshapes_train" ,train_size)
    

In [3]:
# computing the optimal parameters by minimizing the least squares error function
def linear_regression(X, y):
    return np.linalg.pinv(X.T @ X) @ X.T @ y

w = linear_regression(X_train, y_train)

In [10]:
y_test = np.load("orientations_test.npy")
test_size = 1000

X_test = vectorize_images("3dshapes_test" ,test_size)
y_prediction = X_test @ w

RMSE = math.sqrt(np.square(np.subtract(y_test, y_prediction)).mean()) 

'''for i in range(100):
    print(y_test[i], end=' ')
    print(y_prediction[i])
'''

print(RMSE)


0.23033949408208762


In [16]:
# Apply Fourier transform

x_train_extracted = np.zeros(shape=(train_size, 10))

image = cv2.imread('image.jpg', 0)

f = np.fft.fft(X_train[1])
fshift = np.fft.fftshift(f)

print(f[:50])
print(fshift[:50])


magnitude_spectrum = 20*np.log(np.abs(fshift))
    

# print(x_train_extracted)


[ 4.48430000e+05+0.00000000e+00j  8.57993921e+04-1.08401587e+05j
 -1.13050704e+04-8.19191587e+04j -2.40667173e+04-1.40466855e+04j
  1.58424682e+04-5.85701543e+03j  8.22575503e+03-2.67457625e+04j
 -1.43112322e+04-1.93879326e+04j -1.32081944e+03-6.41559031e+03j
  7.71187527e+03-9.90642237e+03j -1.17066772e+02-1.62523213e+04j
 -2.52698187e+03-6.24479399e+03j  7.45918049e+02-2.98716243e+03j
  6.37579424e+02-9.87433090e+03j -1.11355363e+03-7.95738669e+03j
 -8.12553471e+02-5.19618773e+03j  1.61694222e+03-6.51517342e+03j
  2.13115786e+02-4.86541560e+03j -4.10473297e+02-4.76282704e+03j
  4.25716248e+02-5.15693763e+03j -5.92623092e+02-4.29109493e+03j
 -3.93715366e+02-3.88697187e+03j  9.95474252e+02-4.52573704e+03j
 -3.14070100e+02-4.45006690e+03j -4.16450263e+02-3.52696185e+03j
  7.58012765e+02-2.93330076e+03j  1.12345940e+02-3.81001201e+03j
 -4.81897654e+02-3.51338486e+03j -2.80092466e+02-2.59225853e+03j
  4.11995253e+02-3.18278592e+03j  2.23905632e+02-3.56295386e+03j
 -4.26398714e+02-2.769387